# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Импортируем `pandas` и читаем файл в переменную `data`.

In [1]:
import pandas as pd

data = pd.read_csv('./users_behavior.csv')

Импортируем все модели, которые могут понадобиться для работы из библиотеки `sklearn`

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

Импортируем из библиотеки `sklearn` вспомогательные функции для оценки точности модели и разделения датасета на обучающую и валидационную выборку.

In [3]:
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split

Рассмотрим данные:

In [4]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Вывод**

Данные выглядят вполне адекватными, не требующими дополнительной обработки. Пропуски отсутствуют, типы данных указаны верно.

Следовательно, данные можно использовать для обучения моделей.

## Разбейте данные на выборки

Выделим из общих данных две переменных для дальнешей работы, `X` содержащие все признаки (features) и `y` содержащий только целевой признак (target)

In [6]:
X = data.copy()
y = X.pop('is_ultra')

Разделим полученные данные на обучающую, валидационную и тестовую выборки с использованием `train_test_split` из библиотеки `sklearn`.

Первым этапом выделим 25% выборки как тестовые (если не указано другого, значение test_size=0.25)

In [7]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

Далее выделим такие же 25% из оставшихся данных (18.75% от изначального) для валидации, всё оставшееся - для обучения:

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.25, random_state=13)

**Вывод**

Данные подготовлены к моделированию, и разбиты в пропорциях:

- **56.25%** - тренировочные данные
- **25%** - тестовые данные
- **18.75%** - валидационные данные.

## Исследуйте модели

Напишем примитивные функции по проверке разных гиперпараметров моделей для выбора наилучшей.

Для проверки модели **DecisionTreeClassifier**

In [9]:
def check_tree(X, y, X_valid, y_valid, m_depth=10):
#   'DecisionTreeClassifier'
    best_model = None
    best_result = 0
    
    for depth in range(1, (m_depth+1)):
        model = DecisionTreeClassifier(max_depth=depth, random_state=13)
        model.fit(X, y)
        result = accuracy_score(y_valid, model.predict(X_valid))
        if result > best_result:
            best_model = model
            best_result = result
    print('Модель DecisionTreeClassifier,',
          '(depth = ', str(best_model.get_params()['max_depth']), ')',
          'accuracy составляет:', best_result)
    return best_model

Для проверки модели **RandomForestClassifier**

In [10]:
def check_rand(X, y, X_valid, y_valid, m_depth=10, n_est=200):
#   'RandomForestClassifier'
    best_model = None
    best_result = 0
    
    for depth in range(1, (m_depth+1)):
        for est in range(10, (n_est+1), 10):
            model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=13)
            model.fit(X, y)
            result = accuracy_score(y_valid, model.predict(X_valid))
        if result > best_result:
            best_model = model
            best_result = result
        
    print('Модель RandomForestClassifier,',
          '(depth = ', best_model.get_params()['max_depth'], ')',
          '(n_estimators = ', best_model.get_params()['n_estimators'], ')',
          'accuracy составляет:', best_result)
    
    return best_model

Для проверки модели **LogisticRegression**

In [11]:
def check_logi(X, y, X_valid, y_valid):
#   'LogisticRegression':  
    model = LogisticRegression(solver = 'lbfgs', random_state=13)
    model.fit(X, y)

    print('Модель LogisticRegression, accuracy составляет:', accuracy_score(y_valid, model.predict(X_valid)))
    
    return model

Воспользуемся нашими функциями `check_tree()`, `check_random()` и `check_logi()`

In [12]:
model_tree = check_tree(X_train, y_train, X_valid, y_valid)

Модель DecisionTreeClassifier, (depth =  3 ) accuracy составляет: 0.7810945273631841


In [13]:
model_rand = check_rand(X_train, y_train, X_valid, y_valid)

Модель RandomForestClassifier, (depth =  9 ) (n_estimators =  200 ) accuracy составляет: 0.7943615257048093


In [14]:
model_logi = check_logi(X_train, y_train, X_valid, y_valid)

Модель LogisticRegression, accuracy составляет: 0.6948590381426202


**Вывод**

- В результате работы функции мы определили наилучшую модель: RandomForestClassifier с гиперпараметрами глубины - 9 и количества деревьев - 200.
- Модель логистической регрессии показала себя хуже всех в данных условиях, со значением точности в ~0.69
- Модель одиночного дерева показала схожую точность, при более быстром выполненнии (0.78 против 0.79 у RandomForest)

**Дополнительная проверка**

Предположительно, значение колонки `calls` не имеет смысла в обучении модели, так как тариф этого не учитывает, и имеет значение только суммарное количество проговоренных минут, в связи с этим, проверим:

In [15]:
features = ['minutes', 'messages', 'mb_used', 'is_ultra']
X2 = data[features].copy()
y2 = X2.pop('is_ultra')

X2_temp, X2_test, y2_temp, y2_test = train_test_split(X2, y2, random_state=13)
X2_train, X2_valid, y2_train, y2_valid = train_test_split(X2_temp, y2_temp, random_state=13)

model_tree_strip = check_tree(X2_train, y2_train, X2_valid, y2_valid)
model_rand_strip = check_rand(X2_train, y2_train, X2_valid, y2_valid)
model_logi_strip = check_logi(X2_train, y2_train, X2_valid, y2_valid)

Модель DecisionTreeClassifier, (depth =  3 ) accuracy составляет: 0.7810945273631841
Модель RandomForestClassifier, (depth =  8 ) (n_estimators =  200 ) accuracy составляет: 0.7976782752902156
Модель LogisticRegression, accuracy составляет: 0.736318407960199


**Дополнительный вывод**

Исключение колонки `calls` из набора данных оказало незначительное влияние на результат (в пределах погрешности), при этом:

- Модель **LogisticRegression** показала себя заметно лучше (0.73 против 0.69)
- Модель **DecisionTreeClassifier** не показала никаких изменений вовсе.
- В то время как модель **RandomForestClassifier** немного улучшило результат (0.798 против 0.794), но при меньшей глубине дерева (8 против 9), что должно при больших объёмах данных снизить затрачиваемое время.

## Проверьте модель на тестовой выборке

Испытаем нашу модель (**RandomForestClassifier**) на тестовой выборке `X_test`

In [16]:
prediction = model_rand.predict(X_test)
accuracy_score(y_test, prediction)

0.8109452736318408

Точность (accuracy), проверенная на тестовой выборке моделью **RandomForestClassifier**, составляет 0,81. Т.е. 81% совпадений с реальностью из 100% возможных.

19% ошибок - достаточно большое значение, и может быть как связано с неверным выбором признаков для моделирования, так и с малым количеством данных для тренировки.

---

Также проверим нашу модель, не включающую в себя колонку `calls`

In [17]:
prediction_strip = model_rand_strip.predict(X_test.drop(columns='calls'))
accuracy_score(y_test, prediction_strip)

0.8146766169154229

Точность по сравнению с полными данынми изменилась незначительно, и также находится в пределах погрешности.

---

Проверим модель, не выбранную как лучшую, но более быструю (**DecisionTreeClassifier**, с параметрами, определёнными как лучшие в рамках работы функции `check_tree()`):

In [18]:
model_tree.fit(X_train, y_train)
accuracy_score(y_test, model_tree.predict(X_test))

0.7997512437810945

Accuracy, определённая на тестовой выборке в данном случае составляет 0.7998, т.е. почти идентичный результат в 80% процентов, при меньшей нагрузке и времени расчёта.

---

**Вывод**

Модель достигла целевой точности (*>0.75*), и выполняет свою работу с ошибкой в 19% случаев, что тем не менее составляет почти 1/5 всех случаев.

## (бонус) Проверьте модели на адекватность

Для проверки модели на адекватность, сравним предсказанные значения со случайными. Для генерации случайных значений загрузим библиотеку `numpy` 

In [19]:
import numpy as np

... и сгенерируем последовательность методом `np.random.randint()` из случайных значений `0` и `1`. Проверку точности проведём тем же методом, что оценивали результат работы моделей, через `accuracy_score()`.

Для чистоты эксперимента, проведём такую генерацию случайных ответов несколько раз, и выберем наилучшее.

In [20]:
score_random = 0
for _ in range(10):
    predictions_random = np.random.randint(0, 2, X_test.shape[0])
    if score_random < accuracy_score(y_test, predictions_random):
        score_random = accuracy_score(y_test, predictions_random)
        
print('Accuracy случайных результатов составляет:', score_random)

Accuracy случайных результатов составляет: 0.5174129353233831


**Промежуточный вывод**

Принятую модель можно признать адекватной, так как её точность составляет **0.81**, что больше accuracy чисто случайной последовательности, равной **0.49-0.55**

Проведём дополнительную проверку модели на адекватность с использованием **DummyClassifier**

In [21]:
from sklearn.dummy import DummyClassifier

dummy_strat = DummyClassifier(strategy = 'stratified')
dummy_freq = DummyClassifier(strategy = 'most_frequent')

dummy_strat.fit(X_train, y_train)
dummy_freq.fit(X_train, y_train)


print('DummyClassifier с параметром stratified, точность составляет:',
      accuracy_score(y_test, dummy_strat.predict(X_test)))
      
print('DummyClassifier с параметром most_frequent, точность составляет:',
      accuracy_score(y_test, dummy_freq.predict(X_test)))

DummyClassifier с параметром stratified, точность составляет: 0.5808457711442786
DummyClassifier с параметром most_frequent, точность составляет: 0.6865671641791045


**Финальный вывод**

При проверке с использованием **DummyClassifier** (*accuracy = 0.68*), адекватными можно считать только модели **DecisionTreeClassifier** и **RandomForestClassifier**, со значениями *accuracy* в районе **0.8**. 

Модель же **LogisticRegression** к адекватным отнести нельзя, так как точность данной модели совпадает с точностью **DummyClassifier**

## Общий вывод

При выборе и тренировки моделей на данном наборе данных можно заключить, что:

- с использованием модели **RandomForestClassifier**, достигается наибольшая точность (accuracy) **0.81**, но требует больше времени на тренировку модели;
- модель **DecisionTreeClassifier** показывает сопоставимый результат по точности (**0.8** на тестовой выборке), и требует меньше системных ресурсов, при этом, всё ещё удовлетворяя искомой точности ***>0.75***. При больших объёмах данных, и при той же искомой точности данная модель имеет больший практический смысл, и рекомендуется к применению;
- модель **LogisticRegression** показала себя хуже всех, и находится за гранью допустимой точности, в данных условиях не применима.